# Multilabel and Multiclass datasets


### Primero importamos todas las librerias que necesitaremos

In [1]:
from sklearn import preprocessing
from skmultilearn.problem_transform import BinaryRelevance,ClassifierChain,LabelPowerset
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.io import arff
import pandas as pd

### Ahora cargaremos en un array todos los metodos que usaremos para clasificar nuestro problema, un paso comun es usar los algoritmos mas conocidos en sus formas basicas y ver cual es el que se comporta mejor con nuestro dataset 

In [2]:
classifiers = [
    KNeighborsClassifier(20),
    SVC(kernel="linear", C=0.06),
    SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=22),
    RandomForestClassifier(max_depth=22, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(n_estimators=10),
    GaussianNB()]

### Ahora crearemos una funcion que recibira un clasificador 'x' , datos de entrenamiento asi como de testeo testeo y nos regresara su desempeño

In [3]:
def make_classifications(classifier,X_train, y_train,X_test,y_test):
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    return accuracy_score(y_test,predictions)

### Ahora carguemos nuestro dataset y examinemoslo, este dataset se encuentra en : 
https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29
Básicamente el dataset tiene 22 campos numericos que representan mediciones del sonido que hacen algunas especies de ranas, dicho dataset tiene tres categorias a clasificar : Families, Genus y Species, cada una de estas, asu vez, es una multiclase y sus datos son:

__Families__                  
- Bufonidae               
- Dendrobatidae           
- Hylidae 
- Leptodactylidae

__Genus__
- Adenomera 
- Ameerega
- Dendropsophus 
- Hypsiboas 
- Leptodactylus 
- Osteocephalus 
- Rhinella 
- Scinax

__Species__
- AdenomeraAndre 
- AdenomeraHylaedact 
- Ameeregatrivittata 
- HylaMinuta 
- HypsiboasCinerascens 
- HypsiboasCordobae 
- LeptodactylusFuscus 
- OsteocephalusOopha 
- Rhinellagranulosa 
- ScinaxRuber

Y es aqui donde las cosas se vuelven interesantes, para una misma entrada tendremos un tipo de familie, genus y Species y no solo eso, cada una de esta categorias es multiclass, entonces ¿como trabajamos con estos datasets?, bueno primero sera ver el dataset: 

In [4]:
df = pd.read_csv("frogs/Frogs_MFCCs.csv")
print(df.head())

   MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
0       1.0  0.152936 -0.105586  0.200722  0.317201  0.260764  0.100945   
1       1.0  0.171534 -0.098975  0.268425  0.338672  0.268353  0.060835   
2       1.0  0.152317 -0.082973  0.287128  0.276014  0.189867  0.008714   
3       1.0  0.224392  0.118985  0.329432  0.372088  0.361005  0.015501   
4       1.0  0.087817 -0.068345  0.306967  0.330923  0.249144  0.006884   

   MFCCs_ 8  MFCCs_ 9  MFCCs_10    ...     MFCCs_17  MFCCs_18  MFCCs_19  \
0 -0.150063 -0.171128  0.124676    ...    -0.108351 -0.077623 -0.009568   
1 -0.222475 -0.207693  0.170883    ...    -0.090974 -0.056510 -0.035303   
2 -0.242234 -0.219153  0.232538    ...    -0.050691 -0.023590 -0.066722   
3 -0.194347 -0.098181  0.270375    ...    -0.136009 -0.177037 -0.130498   
4 -0.265423 -0.172700  0.266434    ...    -0.048885 -0.053074 -0.088550   

   MFCCs_20  MFCCs_21  MFCCs_22           Family      Genus         Species  \
0  0.057684  0.1186

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7195 entries, 0 to 7194
Data columns (total 26 columns):
MFCCs_ 1    7195 non-null float64
MFCCs_ 2    7195 non-null float64
MFCCs_ 3    7195 non-null float64
MFCCs_ 4    7195 non-null float64
MFCCs_ 5    7195 non-null float64
MFCCs_ 6    7195 non-null float64
MFCCs_ 7    7195 non-null float64
MFCCs_ 8    7195 non-null float64
MFCCs_ 9    7195 non-null float64
MFCCs_10    7195 non-null float64
MFCCs_11    7195 non-null float64
MFCCs_12    7195 non-null float64
MFCCs_13    7195 non-null float64
MFCCs_14    7195 non-null float64
MFCCs_15    7195 non-null float64
MFCCs_16    7195 non-null float64
MFCCs_17    7195 non-null float64
MFCCs_18    7195 non-null float64
MFCCs_19    7195 non-null float64
MFCCs_20    7195 non-null float64
MFCCs_21    7195 non-null float64
MFCCs_22    7195 non-null float64
Family      7195 non-null object
Genus       7195 non-null object
Species     7195 non-null object
RecordID    7195 non-null int64
dtypes: float6

In [6]:
print(df.describe())

          MFCCs_ 1     MFCCs_ 2     MFCCs_ 3     MFCCs_ 4     MFCCs_ 5  \
count  7195.000000  7195.000000  7195.000000  7195.000000  7195.000000   
mean      0.989885     0.323584     0.311224     0.445997     0.127046   
std       0.069016     0.218653     0.263527     0.160328     0.162722   
min      -0.251179    -0.673025    -0.436028    -0.472676    -0.636012   
25%       1.000000     0.165945     0.138445     0.336737     0.051717   
50%       1.000000     0.302184     0.274626     0.481463     0.161361   
75%       1.000000     0.466566     0.430695     0.559861     0.222592   
max       1.000000     1.000000     1.000000     1.000000     0.752246   

          MFCCs_ 6     MFCCs_ 7     MFCCs_ 8     MFCCs_ 9     MFCCs_10  \
count  7195.000000  7195.000000  7195.000000  7195.000000  7195.000000   
mean      0.097939    -0.001397    -0.000370     0.128213     0.055998   
std       0.120412     0.171404     0.116302     0.179008     0.127099   
min      -0.410417    -0.538982    -0

### Ahora que ya conocemos al enemigo, prosigamos a limpiarlo, el unico problema que parece tener este dataset es que sus clases son objetos del tipo object, en este caso son textos, asi que prosigamos a volverlos del tipo categorias para mejorar el performance 

In [7]:
df[["Family","Genus","Species"]] = df[["Family","Genus","Species"]].astype('category')

### Bien, el siguiente paso sera convertir cada clase en una columna binaria para que los algoritmos puedan funcionar 

In [8]:
print(df["Family"].head())

dfFamily =  pd.get_dummies(df["Family"])

print(dfFamily.head())

0    Leptodactylidae
1    Leptodactylidae
2    Leptodactylidae
3    Leptodactylidae
4    Leptodactylidae
Name: Family, dtype: category
Categories (4, object): [Bufonidae, Dendrobatidae, Hylidae, Leptodactylidae]
   Bufonidae  Dendrobatidae  Hylidae  Leptodactylidae
0          0              0        0                1
1          0              0        0                1
2          0              0        0                1
3          0              0        0                1
4          0              0        0                1


### Y lo mismo con las otras dos clases 

In [9]:
dfGenus =  pd.get_dummies(df["Genus"])
dfSpecies =  pd.get_dummies(df["Species"])

### Concatenamos los dataframes: 

In [10]:
dft = pd.concat([df[df.columns[0:22]],dfFamily,dfGenus,dfSpecies],axis="columns")

### Separemos en dos dataframes el dataframe original, a decir, uno que represente los datos y otro las clases 

In [11]:
data = df[df.columns[0:22]]
classes = dft[["Adenomera","AdenomeraAndre","AdenomeraHylaedactylus","Ameerega","Ameeregatrivittata",
               "Bufonidae","Dendrobatidae","Dendropsophus","HylaMinuta","Hylidae","Hypsiboas",
               "HypsiboasCinerascens","HypsiboasCordobae","Leptodactylidae","Leptodactylus",
               "LeptodactylusFuscus","Osteocephalus","OsteocephalusOophagus","Rhinella","Rhinellagranulosa",
               "Scinax","ScinaxRuber"]]


### Ahora generemos los conjuntos de entramiento y testeo 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.30)

### Listo y ¿ahora qué?, bueno, no podemos usar cualquier algoritmo aqui por que fallaria estrepitosamente, ya que tenemos un problema de multilabel, es decir, tenemos que clasificar aqui tres clases en vez de una, veamos un ejemplo de como fallaria cualquier algoritmo 

In [13]:
acc = make_classifications(SVC(kernel="linear", C=0.06),X_train, y_train,X_test,y_test)
print(acc)

ValueError: bad input shape (5036, 22)

### entonces que hacemos, bueno tenemos tres formas de enfrentarnos a estos problemas:

- Binary Relevance
- Classifier Chains
- Label Powerset

y basicamente hacen esto:

__Binary Relevance__:
    este es el divide y venceras, basicamente toma cada posible clase individualmente y la clasifica, luego concatena todo y obtiene el resultado:

Original   
<img src="example1.png">
Lo convierte en:
<img src="example2.png">


__Classifier Chains__
    En esta tecnica primero toma solo una columna de clase y la clasifica, luego toma otra columna de clase para clasificarla pero la primera formara parte de los datos , luego tomara una tercera y las dos primeras formaran parte de los datos, mmm una imagen muestra mejor este proceso

Original   
<img src="example1.png">
Lo convierte en:
<img src="example3.png">
Los datos sombreados en amarillo son los que usariamos para clasificar nuestra clase

__Label Powerset__
    A mi parecer el mas sencillo de entender, aqui intentamos etiquetar las diferentes condiciones en que se generan las clases y creamos clases nuevas

Original   
<img src="example4.png">
Lo convierte en:
<img src="example5.png">
  este metodo es el mas simple, sin embargo, cabe mencionar que requeririamos de tener todas las posibles combinaciones para generar todas las clases posibles


Que metodo es mejor, bueno sinceramente es mejor experimentar todos los casos para ver cual se acomoda a nuestro problema, para python tenemos que la libreria skmultilearn.problem_transform nos regala estos metodos, entonces manos a la obra


In [15]:
print("Binary technique---------------------------------------------------------------------")
results = [make_classifications(BinaryRelevance(classifier),X_train, y_train,X_test,y_test) 
           for classifier in classifiers]
print(results)

print("Chain technique----------------------------------------------------------------------")
results = [make_classifications(ClassifierChain(classifier),X_train, y_train,X_test,y_test) 
           for classifier in classifiers]
print(results)

print("Label power technique----------------------------------------------------------------")
results = [make_classifications(LabelPowerset(classifier),X_train, y_train,X_test,y_test) 
           for classifier in classifiers]
print(results)

Binary technique---------------------------------------------------------------------
[0.9569245020842982, 0.667901806391848, 0.9722093561834182, 0.8503937007874016, 0.8786475220009263, 0.8235294117647058, 0.8022232515053266, 0.50764242704956]
Chain technique----------------------------------------------------------------------
[0.9647985178323297, 0.8809634089856415, 0.9814729041222788, 0.9198703103288559, 0.9249652616952293, 0.8971746178786475, 0.9129226493747105, 0.5673923112552107]
Label power technique----------------------------------------------------------------
[0.9629458082445577, 0.9096804075961094, 0.9874942102825383, 0.9421028253821213, 0.9675775822139879, 0.9217230199166281, 0.7401574803149606, 0.9096804075961094]


### Como observamos , al menos para este datasets, las tecnicas que mejor se comportaron fueron Chain y label, ya aqui es cuestion, dependiendo de nuestro problema tomar una desicion.